In [5]:
!pip install dash 
!pip install dash-html-components 
!pip install dash-core-components 
!pip install dash-table 

In [7]:
!pip install pandas
!pip install numpy

In [34]:
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

g_persons_df = pd.read_csv('./db_persons.csv')[['ID', 'Фамилия', 'Имя', 'Отчество']].values
print(g_persons_df)

def make_dict(g_persons_df):
    list = []
    for i in g_persons_df:
        list.append({'label': str(i[1]) + str(i[2]) + str(i[3]), 'value': i[0]})
    return list

app.layout = html.Div([
    html.Label('Dropdown'),
    dcc.Dropdown(
        options=make_dict(g_persons_df),
        value='MTL'
    ),

], style={'columnCount': 2})

if __name__ == '__main__':
    app.run_server(debug=True)

[[71957 'Горшенин' 'Андрей' 'Константинович']
 [71956 'Агеева' 'Анастасия' 'Сергеевна']
 [71949 'Горяинов' 'Владимир' 'Борисович']
 ...
 [9586 'Соркин' 'Леонид' 'Рафаилович']
 [9584 'Меденников' 'Виктор' 'Иванович']
 [9561 'Старков' 'Константин' 'Евгеньевич']]
